In [1]:
import nltk

In [203]:
import math, sys
#from konlpy.tag import Twitter
class BayesianFilter:
    """ 베이지안 필터 """
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.categories= {} # 카테고리 출현 횟수 기록
        self.cfd = _
        self.train =[]
 
    # 텍스트 문장 학습하기 --- (※3)
    def sfit(self, data):
        """ 텍스트 학습 """
        train_data = []
        for s, c in data:
            for w in s.split():
                train_data.append((c, w))
                if c in self.categories :
                    self.categories[c] += 1
                else:
                    self.categories[c] = 1
                self.words.add(w)
        self.cfd = nltk.ConditionalFreqDist(train_data)

            # 텍스트 단어 학습하기 --- (※3)
    def fit(self, data, c):
        """ 텍스트 학습 """
        for w in data:
            self.train.append((c, w))
            if c in self.categories :
                self.categories[c] += 1
            else:
                self.categories[c] = 1
            self.words.add(w)
            
    # 텍스트 단어 + tag 학습하기 --- (※3)
    def fit_tag(self, data, c):
        """ 텍스트 + tag 학습 """
        for w in nltk.pos_tag(data):
            print(w)
            ''' 동사나 명사만 사용 '''
            #if 'V' not in w[1] and 'N' not in w[1]:
            #    continue
            ''' 숫자는 제거 '''
            if w[1] == 'CD': continue
            self.train.append((c, w))
            if c in self.categories :
                self.categories[c] += 1
            else:
                self.categories[c] = 1
            self.words.add(w)

    def trainModel(self):
        ''' 학습에 사용된 데이터 출력 '''
        #print(self.words)
        ''' 조건부 확률 계산 '''
        self.cfd = nltk.ConditionalFreqDist(self.train)

            # 단어 리스트에 점수 매기기--- (※4)
    def score(self, text, category):
        s_category = sum(self.categories.values())         
        score = math.log( self.categories[category] / s_category )
        words = word_tokenize(text)
        for word in words:
            score += math.log( (self.cfd[category][word]+1) / (self.categories[category] + len(self.words)) )
        return score
    
    # 단어 리스트에 점수 매기기--- (※4)
    def score_tag(self, text, category):
        s_category = sum(self.categories.values())         
        score = math.log( self.categories[category] / s_category )
        '''tagging 추가 '''
        words = nltk.pos_tag(word_tokenize(text)) # tagging added
        for word_tag in words:
            score += math.log( (self.cfd[category][word_tag]+1) / (self.categories[category] + len(self.words)) )
        return score
    
    # 예측하기 --- (※5)
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize 
        score_list = []

        for category in self.categories:
            score = self.score(text, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list
    
    def predict_tag(self, text):
        best_category = None
        max_score = -sys.maxsize 
        score_list = []

        for category in self.categories:
            ''' tagging 정보를 활용한 score 함수 '''
            score = self.score_tag(text, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list

In [204]:
# 영화 local file 읽어서 학습 
''' pos tagging '''
from nltk import word_tokenize

bf = BayesianFilter()

with open('lalaland.txt') as f:
    raw = f.read()

sents = nltk.sent_tokenize(raw)
train_size = int(len(sents)*.9)
for i in sents[:train_size]:
    train_data = word_tokenize(i)
    ''' 태깅 사용하는 fit '''
    bf.fit_tag(train_data, 'lala')
test_lala = []
for i in sents[train_size:]:
    test_lala.append(i)

with open('raiders.txt') as f:
    raw = f.read()

sents = nltk.sent_tokenize(raw)
train_size = int(len(sents)*.9)
for i in sents[:train_size]:
    train_data = word_tokenize(i)
    bf.fit_tag(train_data, 'raiders')
test_raiders = []
for i in sents[train_size:]:
    test_raiders.append(i)

bf.trainModel()
# 예측
print(bf.categories)

test_n = len(test_lala) + len(test_raiders)
correct_n = 0
for i in test_lala:
    pre, scorelist = bf.predict_tag(i)
    if pre == 'lala':
        correct_n += 1
for i in test_raiders:
    pre, scorelist = bf.predict_tag(i)
    if pre == 'raiders':
        correct_n += 1
        
print(correct_n/test_n)


('\ufeffLA', 'NN')
('LA', 'NNP')
('LAND', 'NNP')
('Written', 'NNP')
('by', 'IN')
('Damien', 'NNP')
('Chazelle', 'NNP')
('FADE', 'NNP')
('IN', 'NNP')
('...', ':')
('A', 'DT')
('sun-blasted', 'JJ')
('sky', 'NN')
('.', '.')
('We', 'PRP')
('HEAR', 'VBP')
('radios', 'NNS')
('--', ':')
('one', 'CD')
('piece', 'NN')
('of', 'IN')
('music', 'NN')
('after', 'IN')
('another', 'DT')
('...', ':')
('We', 'PRP')
("'re", 'VBP')
('--', ':')
('1', 'CD')
('EXT', 'NNP')
('.', '.')
('101', 'CD')
('FREEWAY', 'NNP')
('-', ':')
('DAY', 'NN')
('Cars', 'NNS')
('are', 'VBP')
('at', 'IN')
('a', 'DT')
('standstill', 'NN')
('.', '.')
('It', 'PRP')
("'s", 'VBZ')
('a', 'DT')
('horrific', 'NN')
('traffic', 'NN')
('jam', 'NN')
('.', '.')
('Morning', 'VBG')
('rush', 'JJ')
('hour', 'NN')
('.', '.')
('Sun', 'NNP')
('beating', 'VBG')
('down', 'RP')
(',', ',')
('asphalt', 'JJ')
('shimmering', 'VBG')
('in', 'IN')
('the', 'DT')
('heat', 'NN')
('.', '.')
('The', 'DT')
('blown-out', 'JJ')
('downtown', 'NN')
('L.A.', 'NNP')
('sk

('you', 'PRP')
('...', ':')
('I', 'PRP')
('--', ':')
('I', 'PRP')
('just', 'RB')
('--', ':')
('CASTING', 'NNP')
('DIRECTOR', 'NNP')
('One', 'CD')
('second', 'NN')
('.', '.')
('Mia', 'NNP')
('stops', 'NNS')
('.', '.')
('The', 'DT')
('Casting', 'NNP')
('Director', 'NNP')
('waves', 'VBZ')
('the', 'DT')
('Assistant', 'NNP')
('in', 'IN')
('.', '.')
('The', 'DT')
('Assistant', 'NNP')
('scurries', 'NNS')
('in', 'IN')
(',', ',')
('shows', 'VBZ')
('her', 'PRP$')
('boss', 'NN')
('a', 'DT')
('post-it', 'NN')
('.', '.')
('Mia', 'NNP')
('stands', 'VBZ')
('there', 'RB')
(',', ',')
('trying', 'VBG')
('to', 'TO')
('hold', 'VB')
('onto', 'IN')
('the', 'DT')
('tears', 'NNS')
(',', ',')
('hold', 'VBP')
('onto', 'IN')
('the', 'DT')
('emotion', 'NN')
(',', ',')
('as', 'IN')
('the', 'DT')
('Casting', 'NNP')
('Director', 'NNP')
('reads', 'VBZ')
('the', 'DT')
('post-it', 'JJ')
('and', 'CC')
('thinks', 'NNS')
('.', '.')
('CASTING', 'NN')
('DIRECTOR', 'NNP')
('(', '(')
('CONT', 'NNP')
("'D", 'NNP')
(')', ')')
(

('tries', 'VBZ')
('talking', 'VBG')
('to', 'TO')
('a', 'DT')
('pair', 'NN')
('of', 'IN')
('WOMEN', 'NNP')
('--', ':')
('who', 'WP')
('promptly', 'RB')
('ditch', 'VBP')
('her', 'PRP')
('.', '.')
('Seeing', 'VBG')
('she', 'PRP')
("'s", 'VBZ')
('now', 'RB')
('alone', 'RB')
(',', ',')
('a', 'DT')
('YOUNG', 'NNP')
('MAN', 'NNP')
('swoops', 'VBZ')
('in', 'IN')
('to', 'TO')
('hit', 'VB')
('on', 'IN')
('her', 'PRP')
('.', '.')
('She', 'PRP')
('makes', 'VBZ')
('a', 'DT')
('hasty', 'JJ')
('exit', 'NN')
('toward', 'IN')
('the', 'DT')
('bar', 'NN')
('--', ':')
('but', 'CC')
('the', 'DT')
("line's", 'NN')
('obscenely', 'RB')
('long', 'RB')
('.', '.')
('She', 'PRP')
('nears', 'VBZ')
('the', 'DT')
('BATHROOM', 'NNP')
('door', 'NN')
('--', ':')
('but', 'CC')
('a', 'DT')
('COUPLE', 'NNP')
('stands', 'VBZ')
('in', 'IN')
('her', 'PRP$')
('way', 'NN')
(',', ',')
('making', 'VBG')
('out', 'RP')
('.', '.')
('She', 'PRP')
('slips', 'VBZ')
('in', 'IN')
('behind', 'IN')
('them', 'PRP')
('...', ':')
('A13', 'NN

('want', 'VB')
('to', 'TO')
('meet', 'VB')
('anyone', 'NN')
('.', '.')
('LAURA', 'NNP')
('You', 'PRP')
("'ll", 'MD')
('like', 'VB')
('her', 'PRP')
('.', '.')
('SEBASTIAN', 'NNP')
('Uh-huh', 'NNP')
('.', '.')
('Does', 'VBZ')
('she', 'PRP')
('like', 'IN')
('jazz', 'NN')
('?', '.')
('LAURA', 'NNP')
('Probably', 'NNP')
('not', 'RB')
('.', '.')
('SEBASTIAN', 'NNP')
('Then', 'RB')
('what', 'WP')
('are', 'VBP')
('we', 'PRP')
('gon', 'VBG')
('na', 'TO')
('talk', 'VB')
('about', 'IN')
('?', '.')
('LAURA', 'NNP')
('You', 'PRP')
("'ll", 'MD')
('talk', 'VB')
('about', 'IN')
('the', 'DT')
('weather', 'NN')
('.', '.')
('SEBASTIAN', 'NNP')
('Ok.', 'NNP')
('Then', 'RB')
('I', 'PRP')
('have', 'VBP')
('someone', 'NN')
('I', 'PRP')
("'d", 'MD')
('like', 'VB')
('you', 'PRP')
('to', 'TO')
('meet', 'VB')
('.', '.')
('He', 'PRP')
("'s", 'VBZ')
('got', 'VBD')
('a', 'DT')
('face', 'NN')
('tattoo', 'NN')
(',', ',')
('but', 'CC')
('a', 'DT')
('heart', 'NN')
('of', 'IN')
('gold', 'NN')
('.', '.')
('Revision', 'NN

('.', '.')
('Sebastian', 'JJ')
('looks', 'VBZ')
('up', 'RP')
('for', 'IN')
('a', 'DT')
('second', 'JJ')
('--', ':')
('and', 'CC')
('sees', 'VBZ')
('her', 'PRP')
('.', '.')
('They', 'PRP')
('look', 'VBP')
('at', 'IN')
('one', 'CD')
('another', 'DT')
('.', '.')
('Just', 'RB')
('a', 'DT')
('moment', 'NN')
('.', '.')
('Then', 'RB')
('--', ':')
('the', 'DT')
('Boss', 'NNP')
('taps', 'VBZ')
('Sebastian', 'JJ')
('on', 'IN')
('the', 'DT')
('shoulder', 'NN')
('.', '.')
('WE', 'NNP')
('STAY', 'NNP')
('ON', 'NNP')
('Mia', 'NNP')
('as', 'IN')
('she', 'PRP')
('watches', 'VBZ')
('Sebastian', 'JJ')
('rise', 'NN')
('with', 'IN')
('the', 'DT')
('Boss', 'NNP')
('.', '.')
('We', 'PRP')
('just', 'RB')
('see', 'VB')
('the', 'DT')
('Boss', 'NNP')
('talking', 'VBG')
('to', 'TO')
('Sebastian', 'NNP')
(',', ',')
('ca', 'MD')
("n't", 'RB')
('hear', 'VB')
('what', 'WP')
('is', 'VBZ')
('said', 'VBD')
('.', '.')
('Then', 'RB')
(',', ',')
('we', 'PRP')
('get', 'VBP')
('closer', 'JJR')
('--', ':')
('and', 'CC')
('re

...

('out', 'IN')
('of', 'IN')
('the', 'DT')
('tunnel', 'NN')
(',', ',')
('then', 'RB')
('the', 'DT')
('truck', 'NN')
(',', ',')
('its', 'PRP$')
('sides', 'NNS')
('cleaned', 'VBN')
('of', 'IN')
('Nazis', 'NNP')
('.', '.')
('In', 'IN')
('the', 'DT')
('rear', 'NN')
('of', 'IN')
('the', 'DT')
('truck', 'NN')
(',', ',')
('the', 'DT')
('Tough', 'NNP')
('Sergeant', 'NNP')
('is', 'VBZ')
('looking', 'VBG')
('with', 'IN')
('distaste', 'NN')
('back', 'RB')
('at', 'IN')
('the', 'DT')
('tunnel', 'NN')
('.', '.')
('There', 'EX')
('remains', 'VBZ')
('only', 'RB')
('him', 'PRP')
('and', 'CC')
('two', 'CD')
('Armed', 'NNP')
('Nazis', 'NNPS')
('with', 'IN')
('the', 'DT')
('Ark', 'NNP')
('.', '.')
('He', 'PRP')
('sends', 'VBZ')
('these', 'DT')
('two', 'CD')
('climbing', 'VBG')
('up', 'RP')
('over', 'IN')
('the', 'DT')
('top', 'NN')
('of', 'IN')
('the', 'DT')
('truck', 'NN')
('.', '.')
('In', 'IN')
('the', 'DT')
('lead', 'NN')
('car', 'NN')
(',', ',')
('the', 'DT')
('Blond', 'NNP')
('Driver', 'NNP')
('is', '

('their', 'PRP$')
('lips', 'NNS')
('are', 'VBP')
('very', 'RB')
('close', 'RB')
('.', '.')
('MARION', 'NNP')
('No', 'NNP')
('.', '.')
('Then', 'RB')
('again', 'RB')
(',', ',')
('I', 'PRP')
('burned', 'VBD')
('up', 'RP')
('that', 'IN')
('plane', 'NN')
('.', '.')
('INDY', 'NNP')
('You', 'PRP')
('saved', 'VBD')
('my', 'PRP$')
('life', 'NN')
('.', '.')
('MARION', 'NNP')
('And', 'CC')
('you', 'PRP')
('saved', 'VBD')
('mine', 'NN')
('.', '.')
('INDY', 'NNP')
('Seems', 'NNP')
('things', 'NNS')
('have', 'VBP')
('worked', 'VBN')
('out', 'RP')
('kind', 'NN')
('of', 'IN')
('even', 'RB')
('.', '.')
('MARION', 'NNP')
('That', 'WDT')
("'s", 'VBZ')
('the', 'DT')
('way', 'NN')
('I', 'PRP')
('like', 'VBP')
('them', 'PRP')
('.', '.')
('INDY', 'NNP')
('Maybe', 'NNP')
('we', 'PRP')
('should', 'MD')
('consider', 'VB')
('all', 'DT')
('past', 'JJ')
('accounts', 'NNS')
('closed', 'VBD')
('.', '.')
('Marion', 'NNP')
('thinks', 'VBZ')
('about', 'IN')
('this', 'DT')
('a', 'DT')
('long', 'JJ')
('time', 'NN')
('.'

('Shliemann', 'NNP')
('nods', 'NNS')
('and', 'CC')
('continues', 'VBZ')
('to', 'TO')
('stare', 'VB')
('at', 'IN')
('the', 'DT')
('Bantu', 'NNP')
('Wind', 'NNP')
(',', ',')
('as', 'IN')
('does', 'VBZ')
('the', 'DT')
('Captain', 'NNP')
('.', '.')
('The', 'DT')
('Pirate', 'NNP')
('crew', 'NN')
('is', 'VBZ')
('all', 'DT')
('lined', 'VBN')
('across', 'IN')
('the', 'DT')
('bow', 'NN')
('.', '.')
('Towering', 'VBG')
('above', 'IN')
('the', 'DT')
('others', 'NNS')
(',', ',')
('standing', 'VBG')
('on', 'IN')
('the', 'DT')
('rail', 'NN')
(',', ',')
('proud', 'NN')
('and', 'CC')
('defiant', 'NN')
(',', ',')
('is', 'VBZ')
('Katanga', 'NNP')
('.', '.')
('Shliemann', 'NNP')
('looks', 'VBZ')
('at', 'IN')
('the', 'DT')
('Wurrfler', 'NNP')
("'s", 'POS')
('Captain', 'NNP')
('a', 'DT')
('moment', 'NN')
('.', '.')
('SHLIEMANN', 'NNP')
('What', 'WP')
('do', 'VBP')
('you', 'PRP')
('think', 'VB')
(',', ',')
('Captain', 'NNP')
('?', '.')
('THE', 'DT')
('WURRFLER', 'NNP')
("'S", 'POS')
('CAPTAIN', 'NNP')
('(',

('holding', 'VBG')
('his', 'PRP$')
('breath', 'NN')
(',', ',')
('but', 'CC')
('the', 'DT')
('vegetation', 'NN')
('gets', 'VBZ')
('denser', 'RB')
('.', '.')
('Finally', 'RB')
(',', ',')
('it', 'PRP')
('pulls', 'VBZ')
('him', 'PRP')
('off', 'RP')
('the', 'DT')
('periscope', 'NN')
('.', '.')
('The', 'DT')
('sub', 'NN')
('moves', 'NNS')
('on', 'IN')
(',', ',')
('disappearing', 'VBG')
('ahead', 'RB')
('.', '.')
('Indy', 'NNP')
('rises', 'VBZ')
('desperately', 'RB')
('through', 'IN')
('the', 'DT')
('dark', 'JJ')
('water', 'NN')
(',', ',')
('his', 'PRP$')
('hand', 'NN')
('outstretched', 'VBD')
('.', '.')
('Then', 'RB')
(',', ',')
('almost', 'RB')
('simultaneously', 'RB')
(',', ',')
('hand', 'NN')
('and', 'CC')
('head', 'NN')
('hit', 'VBD')
('solid', 'JJ')
('rock', 'NN')
('.', '.')
('But', 'CC')
('no', 'DT')
('air', 'NN')
('.', '.')
('Indy', 'NNP')
('feels', 'NNS')
('along', 'IN')
('the', 'DT')
('ceiling', 'NN')
('of', 'IN')
('rock', 'NN')
('.', '.')
('Nothing', 'NN')
('.', '.')
('It', 'PRP')


('makes', 'VBZ')
('a', 'DT')
('fast', 'JJ')
('break', 'NN')
('from', 'IN')
('the', 'DT')
('shadows', 'NNS')
('of', 'IN')
('the', 'DT')
('tunnel', 'NN')
('to', 'TO')
('the', 'DT')
('protection', 'NN')
('of', 'IN')
('a', 'DT')
('high', 'JJ')
('stack', 'NN')
('of', 'IN')
('supplies', 'NNS')
('.', '.')
('He', 'PRP')
('climbs', 'VBZ')
('the', 'DT')
('back', 'NN')
('of', 'IN')
('the', 'DT')
('stack', 'NN')
(',', ',')
('peeks', 'NN')
('over', 'IN')
('and', 'CC')
('surveys', 'VB')
('the', 'DT')
('area', 'NN')
('.', '.')
('WHAT', 'WP')
('HE', 'NNP')
('SEES', 'NNP')
('In', 'IN')
('addition', 'NN')
('to', 'TO')
('the', 'DT')
('Tabernacle', 'NNP')
(',', ',')
('the', 'DT')
('second', 'JJ')
('train', 'NN')
('tunnel', 'NN')
('entrance', 'NN')
(',', ',')
('and', 'CC')
('all', 'PDT')
('the', 'DT')
('activity', 'NN')
(',', ',')
('Indy', 'NNP')
("'s", 'POS')
('glance', 'NN')
('rests', 'VBZ')
('momentarily', 'RB')
('on', 'IN')
('a', 'DT')
('large', 'JJ')
(',', ',')
('heavy', 'JJ')
('metal', 'NN')
('door',

In [202]:
from sklearn.feature_extraction.text import TfidfVectorizer
data=['I love you so much',
'I love pizza so much',
'I miss you I want you',
'I want pizza']

#vectorizer = TfidfVectorizer(token_pattern=u"(?u)\\b\\w+\\b",min_df=1)
vectorizer = TfidfVectorizer()
vtr_data = vectorizer.fit_transform(data)

vtr_data = vtr_data.toarray()
featurenames = vectorizer.get_feature_names()
print(featurenames)
print(vtr_data)

['love', 'miss', 'much', 'pizza', 'so', 'want', 'you']
[[ 0.5         0.          0.5         0.          0.5         0.          0.5       ]
 [ 0.5         0.          0.5         0.5         0.5         0.          0.        ]
 [ 0.          0.49338588  0.          0.          0.          0.3889911
   0.7779822 ]
 [ 0.          0.          0.          0.70710678  0.          0.70710678
   0.        ]]


In [201]:
from sklearn.feature_extraction.text import TfidfVectorizer
data=['I love dogs',
'I hate dogs and knitting',
'Knitting is my hobby and my passion']

#vectorizer = TfidfVectorizer(token_pattern=u"(?u)\\b\\w+\\b",min_df=1)
vectorizer = TfidfVectorizer(token_pattern=u"(?u)\\b\\w+\\b")
vtr_data = vectorizer.fit_transform(data)

vtr_data = vtr_data.toarray()
featurenames = vectorizer.get_feature_names()
print(featurenames)
print(vtr_data)

['and', 'dogs', 'hate', 'hobby', 'i', 'is', 'knitting', 'love', 'my', 'passion']
[[ 0.          0.51785612  0.          0.          0.51785612  0.          0.
   0.68091856  0.          0.        ]
 [ 0.41779577  0.41779577  0.54935123  0.          0.41779577  0.
   0.41779577  0.          0.          0.        ]
 [ 0.26628951  0.          0.          0.35013871  0.          0.35013871
   0.26628951  0.          0.70027741  0.35013871]]
